In [13]:
!pip install selenium
from selenium import webdriver
import time
import json
# driver = webdriver.Chrome()
# driver.get('https://google.com')
# time.sleep(5)

language = 'HEBM95' # Hebrew
print(language)


HEBM95


You are using pip version 9.0.1, however version 20.2.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [14]:
driver = webdriver.Chrome()
driver.get('https://live.bible.is/bible/'+language+'/ROM/1?audio_type=audio')
time.sleep(5)


#elem = driver.find_element_by_class_name("audio-player")
elem = driver.find_element_by_tag_name("audio")
print(elem.get_attribute('src'))
    
driver.close()

https://content.cdn.dbp-prod.dbp4.org/audio/HEBM95/HBRHMTN1DA/B06___01_Romans______HBRHMTN1DA.mp3?x-amz-transaction=6911317&Expires=1597022786&Signature=EpskxHix1YaWVYputGtdHPnrRfc4N2ijeIiDhF42TOSSA86pTSJJMv~yTQjGKB~i73BHShButi6jq89-GLs-ItFlnHE8xualOVa6fXRvMnAsz2-F0bIVnckK~KgcgNCVsDWVe0DSZkh93OL9lySg2j9U7xsBf0P9itbnb9DpZlVFSrus7ZlB23Qvvo~SoKLxKt2Io6Mxb14qvaiDolqhv3EnL86g2CjqZYYIYmg94TYhVKjyGEGbpnQdzTRYT7efZmhNVlLkRcGQWObND~jn5yWvkYp-Grvi9vbNcYcPYQkbSzVwkH5iYVTZI6G7u4rCzhx3BCkdUH6VyYNbaYFq3Q__&Key-Pair-Id=APKAI4ULLVMANLYYPTLQ


In [15]:
justRomans_json = '[{"book_id":"ROM","book_id_usfx":"RM","book_id_osis":"Rom","name":"罗马书","testament":"NT","testament_order":6,"book_order":46,"book_group":"Pauline Epistles","chapters":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],"hasVideo":false}]'
romans = json.loads(justRomans_json)
print(len(romans))

1


In [16]:
def audio_to_book_name(audio):
    if '.' in audio:
        audio = audio.split('.')[0]
    if audio[0] == 'A' and audio[-4] == 'O':
        testament = 'OT'
    elif audio[0] == 'B' and audio[-4] == 'N':
        testament = 'NT'
    else:
        print('ERROR: CANNOT FIND CHAPTER NAME FOR AUDIO ',audio)
        return
    book_idx = int(audio.split('_')[0][1:])
    while '__' in audio:
        audio = audio.replace('__','_')
    chapter_idx = int(audio.split('_')[1])
    
    book_name = ''
    for b in romans:
        if b['testament'] == testament and b['testament_order'] == book_idx: 
            assert(book_name == '') #only one match
            assert(chapter_idx <= len(b['chapters']))
            book_name = b['book_id']
    return book_name, chapter_idx

In [17]:
import os 
import shutil
!pip install unidecode
from unidecode import unidecode

#os.system('rm -rf ./'+language+'/mp3')
#shutil.rmtree('C:/Users/shery/JupyterProjects/'+language+'_mp3')
#os.rmdir('C:/Users/shery/JupyterProjects/'+language+'_mp3')
#os.system('mkdir ./'+language+'/mp3')
os.mkdir(os.path.join('C:/Users/shery/Hebrew/'+language+'_mp3'))
#os.system('rm -rf ./'+language+'/html')
#os.rmdir('C:/Users/shery/JupyterProjects/'+language+'_html')
#shutil.rmtree('C:/Users/shery/JupyterProjects/'+language+'_html')
#os.system('mkdir ./'+language+'/html')
os.mkdir('C:/Users/shery/Hebrew/'+language+'_html')

print('number of books:', len(romans))
for book in romans:
    book_id = book['book_id']
    chapters = book['chapters']
    print(book_id,' number of chapters: ',len(chapters))
    for c_i in range(1,len(chapters)+1):
        assert c_i == chapters[c_i-1]
        print('https://live.bible.is/bible/'+language+'/'+str(book_id)+'/'+str(c_i)+'?audio_type=audio')
        driver.get('https://live.bible.is/bible/'+language+'/'+str(book_id)+'/'+str(c_i)+'?audio_type=audio')
        time.sleep(5)

        elem = driver.find_element_by_tag_name("audio")
        src = elem.get_attribute('src')
        assert 'mp3' in src
        print(src)
        start = src.index(language)+18
        end = src.index('mp3')+3
        mp3_name = src[start:end] 
        print(mp3_name)
        #print(src[start-1], src[end])
        assert (src[start-1] == '/' and src[end] == '?')
        #assert (not(mp3_name in os.listdir('./'+language+'/mp3/'))
        #wget(src, './'+language+'/mp3/'+mp3_name)
        os.system('wget "'+src+'" -O ./'+language+'_mp3/'+mp3_name)
        
        html_doc = driver.page_source

        html_file = open('C:/Users/shery/Hebrew/'+language+'_html/'+mp3_name[:-4]+'.html',"w")
        html_file.write(unidecode(html_doc))
        html_file.close()
driver.close()

('number of books:', 1)
(u'ROM', ' number of chapters: ', 16)
https://live.bible.is/bible/HEBM95/ROM/1?audio_type=audio


You are using pip version 9.0.1, however version 20.2.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


InvalidSessionIdException: Message: invalid session id


In [8]:
#mode 'eng' or mode 'rom' or mode 'heb': 
#keep all parenthesis

#mode 'cmn': 
#if found a pair of parenthesis and it contains number, remove content
#if found a pair of parenthesis and it contains text only, don't remove content (but actually some are read in the audio)
    #except for 细拉 which takes 4 char!
    #e.g. Read: MAT_1, ROM_10, JER_36; Not read: GEN_8, JDG_8, NUM_4, EZK_35, PSA_7, PSA_89
#if a verse has only one parenthesis (another is in a different verse), remove parenthesis but keep content
    #e.g. Read: 1JN_1, 2CH_24, JHN_4

def clean_text(text, mode = 'eng'):
    text = text.replace('※','')
    text = text.replace(u'\u00A0','')

    text = text.replace('（','(')
    text = text.replace('）',')')
    
    if (mode == 'eng' or mode == 'rom') or mode == 'heb':
        return text
    if mode == 'cmn':
        if '(' in text or ')' in text:    
            #print('parenthesis in text:', text)
            while re.search('\([^()]*[1-9][^()]*\)',text) != None:
                found = re.search('\([^()]*[1-9][^()]*\)',text)
                #print('a pair of parenthesis with number in it:',text[found.start():found.end()])
                text = text[:found.start()]+text[found.end():]
                #print('after removal:',text)
            while re.search('\([^()]*\)',text) != None:
                found = re.search('\([^()]*\)',text)
                print('a pair of parenthesis with only text in it',text)
                if '细拉' in text[found.start():found.end()]: #(细拉); or (士迦庸、细拉) appearing once
                    text = text[:found.start()]+text[found.end():]
                    print('found 细拉, removed')
                else:
                    print('length > r, only remove parenthesis but keep the content')
                    text = text.replace('(','[')
                    text = text.replace(')',']')
                print('after removal:',text)
            if '(' in text or ')' in text: 
                print('single parenthesis:',text)
                text = text.replace('(','')
                text = text.replace(')','')
        return text

In [9]:
#Python3
!pip install bs4
from bs4 import BeautifulSoup
import os
import shutil

language = 'HEBM95'
#get all text in html first, find \u00A0+idx+\u00A0 to anchor the verses

#shutil.rmtree('C:/Users/shery/JupyterProjects/'+language+'_txt_verse_1')
#os.system('rm -rf C:/Users/shery/JupyterProjects/'+language+'_txt_verse_1')
os.mkdir('C:/Users/shery/Hebrew/'+language+'_txt_verse_1')

html_list = os.listdir('C:/Users/shery/Hebrew/'+language+'_html')
for html_name in html_list:
    if 'html' in html_name:
        book_id, c_i = audio_to_book_name(html_name)
        print(html_name, book_id, c_i)
        
        html_file= open('C:/Users/shery/Hebrew/'+language+'_html/'+html_name,"r",encoding='utf-8')
        html_doc = html_file.read().replace(u'\n', u'')
        html_file.close()

        txt_file= open('C:/Users/shery/Hebrew/'+language+'_txt_verse_1/'+html_name[:-5]+'.tsv',"w",encoding='utf-8')
        soup = BeautifulSoup(html_doc, 'html.parser')
        main = soup.find_all('main')
        texts = main[0].find_all(attrs={'class':'justify'})
        assert(len(texts)==1)
        t = texts[0]
        content = t.get_text()
        #print(content)
        assert('\n' not in content)
        
        verse_i = 1
        verse_name = '0'
        while u'\u00A0' in content:
            if u'\u00A0'+str(verse_i)+u'\u00A0' in content:
                head = content.split(u'\u00A0'+str(verse_i)+u'\u00A0')[0]
                content = content.split(u'\u00A0'+str(verse_i)+u'\u00A0')[1]
                #print('verse ',verse_i-1,': ',head)
                head = clean_text(head)
                txt_file.write(verse_name+'\t'+head+'\n')
                verse_name = str(verse_i)
                verse_i += 1
            elif str(verse_i)+u'-'+str(verse_i+1)+u'\u00A0' in content: #e.g. verse is 2-3 is merged 
                print(str(verse_i)+u'-'+str(verse_i+1), ' verses are merged into one')
                head = content.split(str(verse_i)+'-'+str(verse_i+1)+u'\u00A0')[0]
                content = content.split(str(verse_i)+'-'+str(verse_i+1)+u'\u00A0')[1]
                head = clean_text(head)
                txt_file.write(verse_name+'\t'+head+'\n')
                verse_name = str(verse_i)+'-'+str(verse_i+1)
                verse_i += 2
            elif str(verse_i)+u'-'+str(verse_i+2)+u'\u00A0' in content: #e.g. verse is 6-8 is merged 
                print(str(verse_i)+u'-'+str(verse_i+2), ' verses are merged into one')
                head = content.split(str(verse_i)+'-'+str(verse_i+2)+u'\u00A0')[0]
                content = content.split(str(verse_i)+'-'+str(verse_i+2)+u'\u00A0')[1]
                head = clean_text(head)
                txt_file.write(verse_name+'\t'+head+'\n')
                verse_name = str(verse_i)+'-'+str(verse_i+2)
                verse_i += 3      
            elif u'\u00A0'+str(verse_i+1)+u'\u00A0' in content: #one verse is skipped in the text
                print('verse ',verse_i,'is not in the text, skipping to the next verse')
                verse_i += 1
            else:
                if re.search('[0-9]',content) != None:
                    print('OUTLIER!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!',content)
                #the last verse may have \u00a0 in text, need to break loop manually here
                break          
                
        #last verse   
        #print('last verse ',verse_name,': ',content)
        content = clean_text(content)
        txt_file.write(verse_name+'\t'+content+'\n')
        
        txt_file.close()   

B06___01_Romans______HBRHMTN1DA.html ROM 1
B06___02_Romans______HBRHMTN1DA.html ROM 2
B06___03_Romans______HBRHMTN1DA.html ROM 3
B06___04_Romans______HBRHMTN1DA.html ROM 4
B06___05_Romans______HBRHMTN1DA.html ROM 5
B06___06_Romans______HBRHMTN1DA.html ROM 6
B06___07_Romans______HBRHMTN1DA.html ROM 7
B06___08_Romans______HBRHMTN1DA.html ROM 8
B06___09_Romans______HBRHMTN1DA.html ROM 9


You are using pip version 9.0.1, however version 20.2.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


B06___10_Romans______HBRHMTN1DA.html ROM 10
B06___11_Romans______HBRHMTN1DA.html ROM 11
B06___12_Romans______HBRHMTN1DA.html ROM 12
B06___13_Romans______HBRHMTN1DA.html ROM 13
B06___14_Romans______HBRHMTN1DA.html ROM 14
B06___15_Romans______HBRHMTN1DA.html ROM 15
B06___16_Romans______HBRHMTN1DA.html ROM 16
verse  24 is not in the text, skipping to the next verse


In [1]:
# switch kernel to Python2 if on windows
import os
import aeneas
from aeneas.executetask import ExecuteTask
from aeneas.task import Task
language = 'HEBM95'
#os.system('rm -rf ./'+language+'/txt_verse_1_textonly')
os.mkdir('C:/Users/shery/Hebrew/'+language+'_txt_verse_1_textonly')



txt_list = os.listdir('C:/Users/shery/Hebrew/'+language+'_txt_verse_1')
for txt_name in txt_list:
    #print(txt_name)
    textonly = open('C:/Users/shery/Hebrew/'+language+'_txt_verse_1_textonly/'+txt_name[:-3]+'txt','w')
    print('C:/Users/shery/Hebrew/'+language+'_txt_verse_1_textonly/'+txt_name[:-3]+'txt')
    verse_lines = open('C:/Users/shery/Hebrew/'+language+'_txt_verse_1/'+txt_name,'r').readlines()
    for line in verse_lines:
        content = line.split('\t')[1]
        assert(content[-1] == '\n')
        textonly.write(content)
    textonly.close()

#os.system('rm -rf ./'+language+'/syncmap_verse_1_textonly')
os.mkdir('C:/Users/shery/Hebrew/'+language+'_syncmap_verse_1_textonly')

txt_list = os.listdir('C:/Users/shery/Hebrew/'+language+'_txt_verse_1')

short_fragment = 0
total_fragment = 0
for txt_name in txt_list:
    fname = txt_name.split('.')[0]
    print(fname)

    # create Task object
    config_string = u"task_language=heb|is_text_type=plain|os_task_file_format=tsv"
    task = Task(config_string=config_string)
    task.audio_file_path_absolute = 'C:/Users/shery/Hebrew/'+language+'_mp3/'+fname+'.mp3'
    task.text_file_path_absolute = 'C:/Users/shery/Hebrew/'+language+'_txt_verse_1_textonly/'+fname+'.txt'
    task.sync_map_file_path_absolute = 'C:/Users/shery/Hebrew/'+language+'_syncmap_verse_1_textonly/'+fname+'.tsv'

    # process Task
    ExecuteTask(task).execute()

    # output sync map to file
    task.output_sync_map_file()

    for fragment in task.sync_map_leaves():
        total_fragment += 1
        if fragment.length < 5.0:
            short_fragment += 1
            print('short:', fragment, fragment.text)
        if fragment.identifier == 'f000001':
            print('the first:', fragment, fragment.text)
print('% of short segment:', float(short_fragment)/total_fragment)      


C:/Users/shery/Hebrew/HEBM95_txt_verse_1_textonly/B06___01_Romans______HBRHMTN1DA.txt
C:/Users/shery/Hebrew/HEBM95_txt_verse_1_textonly/B06___02_Romans______HBRHMTN1DA.txt
C:/Users/shery/Hebrew/HEBM95_txt_verse_1_textonly/B06___03_Romans______HBRHMTN1DA.txt
C:/Users/shery/Hebrew/HEBM95_txt_verse_1_textonly/B06___04_Romans______HBRHMTN1DA.txt
C:/Users/shery/Hebrew/HEBM95_txt_verse_1_textonly/B06___05_Romans______HBRHMTN1DA.txt
C:/Users/shery/Hebrew/HEBM95_txt_verse_1_textonly/B06___06_Romans______HBRHMTN1DA.txt
C:/Users/shery/Hebrew/HEBM95_txt_verse_1_textonly/B06___07_Romans______HBRHMTN1DA.txt
C:/Users/shery/Hebrew/HEBM95_txt_verse_1_textonly/B06___08_Romans______HBRHMTN1DA.txt
C:/Users/shery/Hebrew/HEBM95_txt_verse_1_textonly/B06___09_Romans______HBRHMTN1DA.txt
C:/Users/shery/Hebrew/HEBM95_txt_verse_1_textonly/B06___10_Romans______HBRHMTN1DA.txt
C:/Users/shery/Hebrew/HEBM95_txt_verse_1_textonly/B06___11_Romans______HBRHMTN1DA.txt
C:/Users/shery/Hebrew/HEBM95_txt_verse_1_textonly/B06_

OSError: File 'C:/Users/shery/Hebrew/HEBM95_mp3/B06___01_Romans______HBRHMTN1DA.mp3' cannot be read

In [1]:
import os
!pip install bs4
from bs4 import BeautifulSoup
import aeneas
from aeneas.executetask import ExecuteTask
from aeneas.task import Task
language = 'HEBM95'

#os.system('rm -rf ./'+language+'/segmented_audio')
os.mkdir('C:/Users/shery/Hebrew/'+language+'_segmented_audio')

txt_list = os.listdir('C:/Users/shery/Hebrew/'+language+'_txt_verse_1')
for txt_name in txt_list:
    print(txt_name)
    alignments = open('C:/Users/shery/Hebrew/'+language+'_syncmap_verse_1_textonly/'+txt_name[:-3]+'tsv','r').readlines()
    verse_lines = open('C:/Users/shery/Hebrew/'+language+'_txt_verse_1/'+txt_name,'r').readlines()
    #if len(alignments) != len(verse_lines):
    #    print('ERROR!!! alignment length ',len(alignments),'verse length ',len(verse_lines))
    assert(len(alignments) == len(verse_lines))
    for i in range(len(verse_lines)):
        verse_id = verse_lines[i].split('\t')[0]
        content = verse_lines[i].split('\t')[1].strip()
        start = alignments[i].split('\t')[0]
        end = alignments[i].split('\t')[1]
        assert(start[-4] == end[-4] == '.') #make sure the time stamp has 3 decimals
        segment_name = txt_name[:-4]+'_'+verse_id+'_'+start.replace('.','')+'_'+end.replace('.','')+'.mp3'
        duration = str(float(end)-float(start))
        cmd = 'sox C:/Users/shery/Hebrew/'+language+'_mp3/'+txt_name[:-3]+'mp3 C:/Users/shery/Hebrew/'+language+'_segmented_audio/'+segment_name+' trim '+start+' '+duration
        #print(cmd)
        os.system(cmd)
print(cmd)        

You are using pip version 9.0.1, however version 20.2.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


B06___01_Romans______HBRHMTN1DA.tsv
B06___02_Romans______HBRHMTN1DA.tsv
B06___03_Romans______HBRHMTN1DA.tsv
B06___04_Romans______HBRHMTN1DA.tsv
B06___05_Romans______HBRHMTN1DA.tsv
B06___06_Romans______HBRHMTN1DA.tsv
B06___07_Romans______HBRHMTN1DA.tsv
B06___08_Romans______HBRHMTN1DA.tsv
B06___09_Romans______HBRHMTN1DA.tsv
B06___10_Romans______HBRHMTN1DA.tsv
B06___11_Romans______HBRHMTN1DA.tsv
B06___12_Romans______HBRHMTN1DA.tsv
B06___13_Romans______HBRHMTN1DA.tsv
B06___14_Romans______HBRHMTN1DA.tsv
B06___15_Romans______HBRHMTN1DA.tsv
B06___16_Romans______HBRHMTN1DA.tsv
sox C:/Users/shery/JupyterProjects/HEBM95_mp3/B06___16_Romans______HBRHMTN1DA.mp3 C:/Users/shery/JupyterProjects/HEBM95_segmented_audio/B06___16_Romans______HBRHMTN1DA_27_196680_210280.mp3 trim 196.680 13.6


In [1]:
# switch kernel to Python3
!pip install nltk
from nltk.tokenize import word_tokenize
import os
language = 'HEBM95'

#os.system('rm -rf ./'+language+'/txt_verse_1_tokenized')
os.mkdir('C:/Users/shery/Hebrew/'+language+'_txt_verse_1_tokenized')


txt_list = os.listdir('C:/Users/shery/Hebrew/'+language+'_txt_verse_1')
for txt_name in txt_list:
    print(txt_name)
    textonly = open('C:/Users/shery/Hebrew/'+language+'_txt_verse_1_tokenized/'+txt_name[:-3]+'txt','w', encoding='utf-8')
    verse_lines = open('C:/Users/shery/Hebrew/'+language+'_txt_verse_1/'+txt_name,'r', encoding='utf-8').readlines()
    for line in verse_lines:
        content = line.split('\t')[1]
        assert(content[-1] == '\n')
        toks = word_tokenize(content.strip())
        line_tok = ' '.join(toks)
        #for p in string.punctuation:
        #    content = content.replace(p,' ')
        #while '  ' in content:
        #    content = content.replace('  ',' ')
        #content = ''.join([c for c in content if c.isalpha() or c == ' ' or c == '\n'])
        #content = content.lower()
        print(line_tok)
        textonly.write(line_tok+'\n')
    textonly.close()
    #†, ¿, á, ñ

You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


B06___01_Romans______HBRHMTN1DA.tsv
1
‏ מֵאֵת שָׁאוּל , עֶבֶד הַמָּשִׁיחַ יֵשׁוּעַ , אֲשֶׁר נִקְרָא לִהְיוֹת שָׁלִיחַ וְיֻחַד לִבְשׂוֹרַת אֱלֹהִים ,
‏ בְּשׂוֹרָה שֶׁאֱלֹהִים הִבְטִיחַ מִקֶּדֶם בְּפִי נְבִיאָיו בְּכִתְבֵי הַקֹּדֶשׁ ,
‏ עַל־אוֹדוֹת בְּנוֹ , יֵשׁוּעַ הַמָּשִׁיחַ אֲדוֹנֵנוּ , שֶׁלְּפִי הַבָּשָׂר מוֹצָאוֹ מִזֶּרַע דָּוִד
‏ וּלְפִי רוּחַ הַקֹּדֶשׁ הֻפְגַּן שֶׁהוּא בֶּן־אֱלֹהִים בִּגְבוּרָה בִּתְחִיָּתוֹ מִן הַמֵּתִים .
‏ דַּרְכּוֹ קִבַּלְנוּ חֶסֶד וּשְׁלִיחוּת לְהָבִיא לִידֵי צִיּוּת שֶׁבָּאֱמוּנָה בְּכָל הָאֻמּוֹת לְמַעַן שְׁמוֹ :
‏ בֵּינֵיהֶם גַּם אַתֶּם קְרוּאֵי יֵשׁוּעַ הַמָּשִׁיחַ .
‏ ¶ וּבְכֵן , אֶל כָּל אֲהוּבֵי אֱלֹהִים בְּרוֹמָא הַנִּקְרָאִים לִהְיוֹת קְדוֹשִׁים : חֶסֶד וְשָׁלוֹם לָכֶם מֵאֵת הָאֱלֹהִים אָבִינוּ וְהָאָדוֹן יֵשׁוּעַ הַמָּשִׁיחַ .
‏ ¶ רֵאשִׁית , אֲנִי מוֹדֶה לֵאלֹהַי עַל כֻּלְּכֶם בְּאֶמְצָעוּת יֵשׁוּעַ הַמָּשִׁיחַ , מִשּׁוּם שֶׁדְּבַר אֱמוּנַתְכֶם נִשְׁמַע בְּכָל הָעוֹלָם .
‏ וְהָאֱלֹהִים אֲשֶׁר אֲנִי עוֹבֵד אוֹתוֹ בְּרוּחִי בְּהַכְרִיזִי אֶת בְּשׂוֹר

B06___16_Romans______HBRHMTN1DA.tsv
16
‏ אֲנִי מַמְלִיץ לִפְנֵיכֶם עַל אֲחוֹתֵנוּ פֶבִּי , שַׁמָּשִׁית הַקְּהִלָּה בְּקֶנְכְּרֶאָה .
‏ קַבְּלוּ נָא אוֹתָהּ בָּאָדוֹן , כַּיָּאֶה לַקְּדוֹשִׁים , וְעִזְרוּ לָהּ בְּכָל דָּבָר שֶׁתִּזְדַּקֵק לָכֶם : כִּי אָכֵן הָיְתָה עֵזֶר לְרַבִּים וְגַם לִי .
‏ דִּרְשׁוּ בִּשְׁלוֹם פְּרִיסְקִילָה וַעֲקִילַס חֲבֵרַי לַעֲבוֹדָה בַּמָּשִׁיחַ יֵשׁוּעַ ,
‏ אֲשֶׁר סִכְּנוּ אֶת רֹאשָׁם בְּעַד נַפְשִׁי , וְלֹא רַק אֲנִי מַכִּיר לָהֶם טוֹבָה , אֶלָּא גַּם כָּל קְהִלּוֹת הַגּוֹיִם .
‏ וּדְרִישַׁת שָׁלוֹם לַקְּהִלָּה שֶׁבְּבֵיתָם . דִּרְשׁוּ בִּשְׁלוֹם אֶפֵּינֶטוֹס יַקִּירִי , שֶׁהָיָה פְּרִי רִאשׁוֹן לַמָּשִׁיחַ בִּמְחוֹז אַסְיָה .
‏ דִּרְשׁוּ בִּשְׁלוֹם מִרְיָם , שֶׁעָמְלָה רַבּוֹת בִּשְׁבִילְכֶם .
‏ דִּרְשׁוּ בִּשְׁלוֹם אַנְדְּרוֹנִיקוֹס וְיוּנְיָס , בְּנֵי־עַמִּי שֶׁהָיוּ אִתִּי בַּמַּאֲסָר : נִכְבָּדִים הֵם בְּקֶרֶב הַשְּׁלִיחִים וְגַם הָיוּ בַּמָּשִׁיחַ לְפָנַי .
‏ דְּרִישַׁת שָׁלוֹם לְאַמְפְּלִיאָטוֹס יַקִּירִי בָּאָדוֹן .
‏ דְּרִישַׁת שָׁלוֹם לְאוּרְבָּנוֹ

In [ ]:
import os
language = 'RONCORN'
path = 'C:\Users\shery\Hebrew\RONCORN_segmented_audio'
file1 = open("SegmentNames.txt", "w")  
txt_list = os.listdir(path)
for txt_name in txt_list:
    if 'B06' in txt_name:
        print(txt_name)
        file1.write(txt_name + '\n')